# Wan2GP on Google Colab

Sets up [Wan2GP](https://github.com/deepbeepmeep/Wan2GP) in a fresh GPU-backed Colab session.

Run the cells in order to prepare the runtime, install dependencies, and launch the Gradio interface. Click on the link in the output from the last cell to launch the app in your browser.

> **Colab VRAM note:** the free tier usually assigns a 15 GB T4 GPU. Most Wan2GP models exceed that budget; the Wan 2.2 TextImage2Video FastWan model works, producing roughly a 5 second 480p clip in about 8 minutes.


## 1. Confirm the accelerator

Choose `Runtime → Change runtime type` and select **GPU** before running anything else.

If this cell raises an error, go back to `Runtime → Change runtime type`, pick **GPU** and save.


In [1]:
import subprocess

try:
    subprocess.run(['nvidia-smi'], check=True)
except Exception as exc:
    raise RuntimeError(
        'GPU not detected. In Colab, open Runtime → Change runtime type, select GPU (or TPU if GPUs are unavailable), save, then rerun this cell.'
    ) from exc


## 2. Configure the workspace path

Choose where Wan2GP should be installed. Update `WAN2GP_ROOT` if you prefer a different location.


In [2]:
from pathlib import Path

WAN2GP_ROOT = Path('/content/Wan2GP').resolve()
print(f'Wan2GP will be installed to: {WAN2GP_ROOT}')


Wan2GP will be installed to: /content/Wan2GP


## 3. Download or update Wan2GP

Clone the repository if it is not present yet; otherwise pull the latest changes.


In [3]:
import subprocess

repo_url = 'https://github.com/deepbeepmeep/Wan2GP.git'
if WAN2GP_ROOT.exists():
    print('Repository already exists. Pulling latest changes...')
    subprocess.run(['git', '-C', str(WAN2GP_ROOT), 'pull'], check=True)
else:
    subprocess.run(['git', 'clone', repo_url, str(WAN2GP_ROOT)], check=True)


## 4. Install system dependencies

Install shared libraries needed for video and audio processing. If you see a warning about skipping an extra repository, it is safe to ignore.

In [4]:
import os, subprocess

env = os.environ.copy()
env['DEBIAN_FRONTEND'] = 'noninteractive'

subprocess.run(['sudo', 'apt-get', 'update', '-qq'], check=True, env=env)
subprocess.run([
    'sudo', 'apt-get', 'install', '-y', '--no-install-recommends',
    'ffmpeg', 'libglib2.0-0', 'libgl1', 'libportaudio2'
], check=True, env=env)


CompletedProcess(args=['sudo', 'apt-get', 'install', '-y', '--no-install-recommends', 'ffmpeg', 'libglib2.0-0', 'libgl1', 'libportaudio2'], returncode=0)

## 5. Install Python dependencies

Install PyTorch, xformers and Wan2GP's Python packages.

In [5]:
import os, subprocess, sys

env = os.environ.copy()
env.setdefault('DEBIAN_FRONTEND', 'noninteractive')

subprocess.run([sys.executable, '-m', 'pip', 'install', '--upgrade', 'pip', 'setuptools', 'wheel'], check=True, env=env)
subprocess.run([sys.executable, '-m', 'pip', 'install', 'torch==2.8.0', 'torchvision', 'torchaudio', '--index-url', 'https://download.pytorch.org/whl/cu128'], check=True, env=env)
subprocess.run([sys.executable, "-m", "pip", "install", "xformers==0.0.32.post2", "--index-url", "https://download.pytorch.org/whl/cu128"],
               check=True, env=env)
subprocess.run([sys.executable, '-m', 'pip', 'install', '-r', str(WAN2GP_ROOT / 'requirements.txt')], check=True, env=env)


CompletedProcess(args=['/usr/bin/python3', '-m', 'pip', 'install', '-r', '/content/Wan2GP/requirements.txt'], returncode=0)

## 5b. Force a headless matplotlib backend

Ensure Wan2GP's preprocessing tools use the headless Agg backend so Step 6 launches cleanly in Colab.


In [6]:
from pathlib import Path

# Replace the TkAgg backend with the headless Agg backend if present.
target = WAN2GP_ROOT / 'preprocessing/matanyone/tools/interact_tools.py'
needle = "matplotlib.use('TkAgg')"
replacement = "matplotlib.use('Agg')"

if not target.exists():
    print(f'Skipping: {target} not found.')
else:
    text = target.read_text()
    if replacement in text:
        print('Agg backend already set; no change needed.')
    elif needle in text:
        target.write_text(text.replace(needle, replacement, 1))
        print('Replaced TkAgg with Agg in interact_tools.py.')
    else:
        print('Backend call not found; no change made.')


Replaced TkAgg with Agg in interact_tools.py.


## 6. Launch Wan2GP

Run the Gradio interface. You will find the gradio link in the output. Click on the link to access the UI. Keep the cell running to stay connected; stop it with the square **Stop** button when you are finished.

In [7]:
import os, subprocess, sys, threading, time

env = os.environ.copy()
env.setdefault('WAN_CACHE_DIR', str(WAN2GP_ROOT / 'models'))
cmd = [
    sys.executable,
    '-u',
    'wgp.py',
    '--listen',
    '--server-port', '7860',
    '--share',
    '--profile', '5',
]
print('Launching Wan2GP…')
process = subprocess.Popen(
    cmd,
    cwd=str(WAN2GP_ROOT),
    env=env,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1,
)
stop_event = threading.Event()

def keepalive():
    while not stop_event.is_set():
        time.sleep(45)
        if stop_event.is_set():
            break
        print('[keepalive] Notebook cell still running…')

keepalive_thread = threading.Thread(target=keepalive, daemon=True)
keepalive_thread.start()

try:
    for line in iter(process.stdout.readline, ''):
        if not line:
            break
        print(line, end='')
except KeyboardInterrupt:
    print('Stopping Wan2GP…')
    process.terminate()
finally:
    stop_event.set()
    process.wait()
    keepalive_thread.join(timeout=1)
    print(f'Wan2GP stopped (return code: {process.returncode}).')


[keepalive] Notebook cell still running…
Stopping Wan2GP…
Wan2GP stopped (return code: -15).


In [8]:
# تحميل النسخة الدقيقة FP8 GGUF بـ modelVersionId 2668710
import os

token = "1c94c6d4d7a02598a51dff7e3b70f0b4"
version_id = "2668710"  # ده الـ ID الصح من الـ link بتاعك

file_name = "wan22_enhanced_nsfw_lightning_fp8_v2668710.gguf"  # اسم واضح عشان نميزه

download_url = f"https://civitai.com/api/download/models/{version_id}?type=Model&format=GGUF&size=full&fp=fp8&token={token}"

# مجلد الـ models في Wan2GP
os.makedirs("/content/Wan2GP/models/unet/", exist_ok=True)

# تثبيت aria2 لو مش موجود (أسرع ويدعم resume)
!apt update -qq && apt install -y aria2 -qq

# التحميل
print("بدء التحميل... الملف كبير (~8-12 GB)، انتظري 5-30 دقيقة حسب السرعة.")
!aria2c --console-log-level=info -c -x 16 -s 16 -k 1M "{download_url}" -d /content/Wan2GP/models/unet/ -o {file_name}

# لو aria2 error، جربي wget كبديل (أبطأ شوية بس موثوق):
# !wget --header="Authorization: Bearer {token}" "{download_url}" -O /content/Wan2GP/models/unet/{file_name}

print(f"✅ الموديل اتحمل! اسمه: {file_name}")
print(f"المسار الكامل لـ Gradio: /content/Wan2GP/models/unet/{file_name}")
print("في Gradio UI: اختاري Custom GGUF / UNET path وحطي المسار ده.")
print("Settings موصى: Steps 4-6, CFG 1, Sampler Euler simple, Resolution 480p, I2V mode مع صورة NSFW high-res.")

87 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
The following additional packages will be installed:
  libaria2-0 libc-ares2
The following NEW packages will be installed:
  aria2 libaria2-0 libc-ares2
0 upgraded, 3 newly installed, 0 to remove and 87 not upgraded.
Need to get 1,513 kB of archives.
After this operation, 5,441 kB of additional disk space will be used.
Selecting previously unselected package libc-ares2:amd64.
(Reading database ... 121858 files and directories currently installed.)
Preparing to unpack .../libc-ares2_1.18.1-1ubuntu0.22.04.3_amd64.deb ...
Unpacking libc-ares2:amd64 (1.18.1-1ubuntu0.22.04.3) ...
Selecting previously unselected package libaria2-0:amd64.
Preparing to unpack .../libaria2-0_1.36.0-1_amd64.deb ...
Unpacking libaria2-0:amd64 (1.36.0-1) ...
S